![dpp](./img/examples.png)

# Transportation Problem

--------------------
* Developed by _Keivan Tafakkori_
* Date: 19-4-2023
* Contact: https://www.linkedin.com/in/keivan-tafakkori/
* GitHub: https://github.com/ktafakkori
--------------------

### Packages

In [9]:
from feloopy import *

### Setting

In [10]:
problem_name = 'ap'
solution_method = 'exact'
interface = 'pulp'
solver = 'highs'
key = 0

### Model

In [11]:
#Environment
m = target_model(solution_method, problem_name, interface, key=key)

#Sets
I = m.set(5) # set of loactions
L = m.set(2) # set of capacity levels

#Dataset
dc = load_from_excel('data.xlsx', [I,I], [1,1], ['i', 'j'], 'driver_cost') # pairwise driver cost
fc = load_from_excel('data.xlsx', [I,I], [1,1], ['i', 'j'], 'freight_cost') # pairwise freight cost
vc = fc/1000 + dc # pairwise variable cost
ec = load_from_excel('data.xlsx', [I,L], [1,1], ['i', 'j'], 'fixed_cost') # establishment cost per capacity level
ca = load_from_excel('data.xlsx', [I,L], [1,1], ['i', 'j'], 'capacity') # capacity
d = load_from_excel('data.xlsx', [I], [1], ['i'], 'demand') # demand

#Variables
x = m.pvar('flow',[I,I])
y = m.bvar('establish',[I,L])

#Objective
m.obj(sum(ec[i,lv]*y[i,lv] for i,lv in sets(I,L)) + sum(vc[i,j] * x[i,j] for i,j in sets(I,I)))

#Constraints
for j in I: 
    m.con(sum(x[i,j] for i in I if i!=j) == d[j])

for i in I:
    m.con(sum(x[i,j] for j in I if j !=i) <= sum(ca[i,lv]*y[i,lv]*1000 for lv in L))
    
#Solve
m.sol(['min'], solver)

### Result

In [12]:
#Report
m.report()

#Display
for i,j in sets(I,I):
    if m.get(x[i,j])>0:
        print(f" transporting {m.get(x[i,j])} units from location {i} to {j} is necessary to meet demand.")

for i,lv in sets(I,L):
    if m.get(y[i,lv])>0:
        print(f"establishing facility of level {lv} at location {i} is necessary to meet demand.")


+--------------------------------------------------------------------------------+
|                                 FelooPy v0.2.5                                 |
+--------------------------------------------------------------------------------+
| Date: 2023-05-27                                                Time: 19:08:29 |
| Interface: pulp                                                  Solver: highs |
+--------------------------------------------------------------------------------+
|                               Model Information                                |
+--------------------------------------------------------------------------------+
|                              The 'ap' model has:                               |
|                    25 positive variable(s) in 1 class(es).                     |
|                     10 binary variable(s) in 1 class(es).                      |
|                                1 objective(s).                                 |
|  